# OpenVINO DL Workbench: Знакомимся с Deep Learning и пишем своё первое AI приложение

СЮДА КАРТИНКУ - ПРЕВЬЮ ИТОГОВОГО РЕЗУЛЬТАТА

## 1. [Introduction](#intro)


## 2. [OpenVINO Overview](#OV-overview)

## 3. [OpenVINO Deep Learning Workbench](#DL-WB-overview)

## 4. [Learn OpenVINO API](#OV-API)

## 5. [Practice](#tasks)

# 1. Introduction  <a name="intro"></a>

### Workshop Contributors

<figure style="display: inline-block;">
  <img style="border-radius: 50%;" src="pictures/Demidovskij.jpg" width="150" height="150"/>
  <figcaption style="text-align: center; font-weight: bold;">Alexander Demidovskij</figcaption>
  <figcaption style="text-align: center">Lead</figcaption>
  <figcaption style="text-align: center">
    <a href="https://github.com/demid5111"><img src="https://github.githubassets.com/favicons/favicon.svg" width="22px"></a>
  </figcaption>
</figure>

<figure style="display: inline-block;">
  <img style="border-radius: 50%;" src="pictures/Tarkan.jpg" width="150" height="150"/>
  <figcaption style="text-align: center; font-weight: bold;">Yaroslav Tarkan</figcaption>
  <figcaption style="text-align: center">Full-Stack Comrade</figcaption>
  <figcaption style="text-align: center">
    <a href="https://github.com/yatarkan"><img src="https://github.githubassets.com/favicons/favicon.svg" width="22px"></a>
  </figcaption>
</figure>

<figure style="display: inline-block;">
  <img style="border-radius: 50%;" src="pictures/Tugaryov.jpg" width="150" height="150"/>
  <figcaption style="text-align: center; font-weight: bold;">Artyom Tugaryov</figcaption>
  <figcaption style="text-align: center">Server-Side Comrade</figcaption>
  <figcaption style="text-align: center">
    <a href="https://github.com/artyomtugaryov"><img src="https://github.githubassets.com/favicons/favicon.svg" width="22px"></a>
  </figcaption>
</figure>

<br>

<figure style="display: inline-block; margin-left: 160px;">
  <img style="border-radius: 50%;" src="pictures/Kashchikhin.jpg" width="150" height="150"/>
  <figcaption style="text-align: center; font-weight: bold;">Andrej Kashchikhin</figcaption>
  <figcaption style="text-align: center">QA Comrade</figcaption>
  <figcaption style="text-align: center">
    <a href="https://github.com/akashchi"><img src="https://github.githubassets.com/favicons/favicon.svg" width="22px"></a>
  </figcaption>
</figure>

<figure style="display: inline-block;">
  <img style="border-radius: 50%;" src="pictures/Savina.jpg" width="150" height="150"/>
  <figcaption style="text-align: center; font-weight: bold;">Tatiana Savina</figcaption>
  <figcaption style="text-align: center">Docs Comrade</figcaption>
  <figcaption style="text-align: center">
    <a href="https://github.com/tsavina"><img src="https://github.githubassets.com/favicons/favicon.svg" width="22px"></a>
  </figcaption>
</figure>

### In This Workshop

Welcome to the Deep Learning workshop where you will find out how to start working with pre-trained neural networks and write your AI application. For that, we will be using OpenVINO™ framework and its graphical interface Deep Learning Workbench. 

During this workshop you will:

1. Learn the basics of neural model analysis and optimization:
    - what a model is and how it works
    - how to measure its performance and analyze the quality
    - how to tune the model for enhanced performance
2. Write your own AI application that detects faces and recognizes emotions on video

### The Deep Learning Era

Deep Learning recently has gained wide popularity due to significant breakthroughs in the artificial neural networks area. Deep Learning has enabled the development of various revolutionary AI applications. With Deep Learning, the algorithm does not need to be taught about the essential features. It can discover features from data on its own using a neural network. Generally, Deep Learning algorithms use a massive amounts of data and aim to simulate the human brain capacity to observe, learn, and make decisions, particularly for highly complex tasks. 

---------------

<figure style="display: inline-block;">
  <img src="pictures/nn-vs-humans-contrast.png" width="400" height="400"/>
  <figcaption style="text-align: center; font-weight: bold;">Classification of images with high contrast</figcaption>
  <figcaption style="text-align: center">
    <a href="https://arxiv.org/pdf/1706.06969.pdf">Source</a>
  </figcaption>
</figure>

<figure style="display: inline-block;">
  <img src="pictures/nn-vs-humans.png" width="400" height="400"/>
  <figcaption style="text-align: center; font-weight: bold;">Humans vs. Neural Models the LFW (Labeled Faces in the Wild) dataset</figcaption>
  <figcaption style="text-align: center">
    <a href="https://arxiv.org/pdf/1404.3840.pdf">Source</a>
  </figcaption>
</figure>

--------------

To perform a specific AI task, the model is firstly trained on the known data. During training, the model makes predictions about what the data represents. Any error in the prediction is used to strengthen the artificial neuron connections until the acceptable accuracy level is achieved. The process of training usually happens only once, requires massive amount of data and powerful computing systems. 

The process of using a pre-trained model to make predictions against previously unseen data is called inference. Inference is performed on different devices, happens several times, and consists in feeding the images to the model. In this workshop we will work with model inference, in other words, we will execute the model already pre-trained to perform useful work.

![](pictures/deep-learning.png)

Author: Mark Robins, Intel, [Link](https://www.intel.ru/content/www/ru/ru/artificial-intelligence/posts/deep-learning-training-and-inference.html)

Let's take a look at the inference on the real-life example of the vehical detection model. The model running alongside the traffic camera is continuously processing a video to identify the cars approaching the intersection. 
1. When a car enters at the red light, several photos of this car are taken and provided to the model, which locates a license plate on the image and sends it for further processing.
2. At the server, a first inference is run to localize the license plate in the image, and a second inference is run to read the characters on the license plate. 
3. After this, the license plate information is sent to the data center, where an application checks for potential traffic violations. 

| Device| Data transmission costs | Power consumption|Сomputing resource|
|:----- | :----- | :----- | :-----|
|Traffic Camera   | Low| Low | Low |
| Gateway Server| Average| Average| Average |
|Data Center | High| High| Powerful|



![](pictures/system.png)

Author: Mark Robins, Intel, [Link](https://www.intel.ru/content/www/ru/ru/artificial-intelligence/posts/deep-learning-training-and-inference.html)

Therefore, given the specific task and the cost of data transmission, it is often beneficial to bring the model inference closer to the target where the data was primarily received. In this case, the pre-trained model requires some accelerating modifications. So let's find out how the OpenVINO™ toolkit can help in achieving better performance of the Deep Learning models. 

# 2. OpenVINO™ Toolkit <a name="OV-overview"></a>

1. [How OpenVINO advances AI technologies](#customers) 
2. [Introduction to the Openvino toolkit](#OV-intro) 
3. [OpenVINO capabilities](#OV-capabilities)
4. [OpenVINO components](#OV-components)
5. [OpenVINO Open Model Zoo](#OMZ)

## 2.1 How OpenVINO advances AI technologies <a name="customers"></a>

The OpenVINO™ toolkit is a comprehensive toolkit for optimizing pretrained deep learning models of various use cases to achieve high performance and prepare them for deployment on Intel® platforms. Based on latest generations of artificial neural networks, including Convolutional Neural Networks (CNNs), recurrent and attention-based networks, the toolkit extends computer vision and non-vision workloads across Intel® hardware, maximizing performance. It accelerates applications with high-performance, AI and deep learning inference deployed from edge to cloud.

SALES SLIDE - 

## 2.2 Introduction to the OpenVINO™ toolkit <a name="OV-intro"></a>

![](pictures/about_vino.png)

## 2.3 OpenVINO™ Capabilities <a name="OV-capabilities"></a>

![](pictures/openvino_toolkit.png)

## 2.4 OpenVINO™ Toolkit Components <a name="OV-components"></a>

![](pictures/additional_tools.png)

## 2.5 OpenVINO™ Open Model Zoo <a name="OMZ"></a>

A model is a network that has been trained over a set of data using a certain framework. Since Deep learning technologies are used in various industrial
applications, it is crucial to have an effective solution for each specific use case. OpenVINO Open Model Zoo provides a range of public and Intel pre-trained models to resolve a variety of different tasks, such as classification, object detection, segmentation and many others.

![](pictures/models.png)

# 3. Deep Learning Workbench: OpenVINO™ Quickstart <a name="DL-WB-overview"></a>

1. [DL Workbench Capabilities](#DL-WB-capabilities)
2. [DL Workbench Workflow](#DL-WB-workflow)

Deep Learning Workbench (DL Workbench) is the official OpenVINO™ graphical interface designed to make the production of pretrained deep learning models significantly easier.
With DL Workbench you can start working with your deep learning model right from your browser: import a model, analyze its performance and accuracy, visualize the outputs, optimize and prepare the model for deployment in a matter of minutes. 


![](pictures/openvino_toolkit-dl-wb-highlighted.png)

## 3.1 DL Workbench Capabilities <a name="DL-WB-capabilities"></a>

<img src="pictures/diagrams_wb_new1.png" width="1050" />


## 3.2 DL Workbench Workflow <a name="DL-WB-workflow"></a>

1. [Open DL Workbench](#open-wb)
2. [Import the Model](#import-model)
3. [Import the Dataset](#import-dataset) 
4. [Benchmark the Model](#model-inference)
5. [Analyze the Model](#analyze-model) 
6. [Optimize the Model](#optimize-model)
7. [Profile the Model](#profile-model)

In [ ]:
# Display button for opening DL Workbench
from IPython.display import HTML

HTML("""<button class="wb-button" onclick="window.open(location.origin, '_blank');">Open DL Workbench</button><style>.wb-button { display: flex; width: fit-content; margin: 20px auto; align-items: center; height: 50px; font-size: 18px; font-weight: 400; font-family: inherit; line-height: 20px; background-color: #003C71; border: 1px solid #003C71; color: #ffffff; cursor: pointer; border-radius: 4px; padding: 0 30px; box-shadow: 0px 4px 10px rgba(0, 0, 0, 0.15);}</style>""")


### Open DL Workbench <a name="open-wb"></a>

To start working with DL Workbench, click **Create** button to open the Create Project page.

![](pictures/create_project_first_page.png)

### Import the Model <a name="import-model"></a>

The first step is to import our model. Select and import face-detection-adas-0001 model from the Open Model Zoo. 

![](pictures/Import_model_button.png)

![](pictures/Import_Model.png)

### Import Dataset <a name="import-dataset"></a>

To work with the model, you need to obain the data. The data can be in different formats depending on the task the model is trained to perform. You can learn more about these formats in the [documentation](https://docs.openvinotoolkit.org/latest/workbench_docs_Workbench_DG_Dataset_Types.html). 

In our case, we will take a set of images and use them as the validation dataset. 
1. Use the following link to download the dataset: [Link](https://github.com/dl-wb-experiments/face-hiding-workshop/files/7043878/dataset.zip).
2. Unarchive it;
3. Go to DL Workbench.

NORMAL DATASET!!!!

![](pictures/dataset_select.png)

Drag and drop the images from the archive to create the dataset and click **Import**.

![](pictures/custom_dataset.png)

### Benchmark the Model <a name="model-inference"></a>

We have imported our model and now want to check how fast it is. For that, let's create our first project. Select the model and the dataset by clicking on them. You can also choose a hardware accelerator on which a model will be executed. We will analyze how our model works on CPU since we have only this device available. 

![](pictures/Project_configured.png)


### Analyze the Model <a name="analyze-model"></a>

When the inference stage is finished, we can see the result that we get after running our model on the CPU. Latency is the time required to process one image. The lower the value, the better. Throughput is the number of images (frames) processed per second. Higher throughput value means better performance. Let's check how the model works and try to make it even faster.

![](pictures/analyze.png)

To test how the model works, you need to select an image from the downloaded dataset, upload this image and visualize the model predictions. 



![](pictures/predictions.png)

### Optimize the Model <a name="optimize-model"></a>

One of the recommended ways to accelerate your model performance is to perform 8-bit integer (INT8) calibration.
Calibration is the process of lowering the precision of a model from floating-point precision (for example, FP32 or FP16) to integer precision (INT8).
A model in the INT8 precision takes up less memory and has higher throughput capacity. INT8 calibration is a universal method to speed up the model.

#### Quantize the Model to Low Precision

![](pictures/quantization.PNG)

#### Post-Trainig Optimization Toolkit


In OpenVINO toolkit INT8 calibration is performed by Post-Training Optimization Tool (POT). POT is used to calibrate a model and then execute it in the INT8 precision.


![](pictures/calibration-int8.png)


![](pictures/int8-page.png)

#### Analyze the Improvements

![](pictures/dashboard-parent-vs-optimized.png)

### Profile the Model <a name="profile-model"></a>

You can observe that after calibration the model has become faster. You can speed up your model even further if you set the parameters specific to each accelerator: streams and batches. In simple terms, streams is the number of cores and batches is the amount of images fed to the model. 

<img src="pictures/batch_stream_diagram.png" width="1050" />



![](pictures/explore-inference.png)

![](pictures/inference-table.png)

After ensuring that our settings accelerated the model, let's use this information to create a working application.

Let’s recap briefly what you have learned at this stage:

1. What a model is and how it works
2. How to measure its performance
3. How to accelerate the model using INT8 calibration
4. How sets of streams and batches affect the model performance

# 4. Learn OpenVINO™ API <a name="OV-API"></a>

Let's learn how to infer a model of object detection use case with OpenVINO™ Python interface and build our application. Object Detection in Computer Vision is a task of finding objects of a certain class and highlighting them with bounding boxes.

We will go through the following steps:

1. [Import required modules](#Import-Required-Modules) 
2. [Configure inference: path to a model and other data](#Configure-an-Inference)
3. [Initialize the OpenVINO™ runtime](#Initialize-the-OpenVINO™-Runtime)
4. [Read the model](#Read-the-Model)
5. [Make the model executable](#Make-the-Model-Executable)
6. [Load the Model to the Device](#Load-to-Device)
7. [Prepare an image for model inference](#Prepare-an-Image-for-Model-Inference)
8. [Infer the model](#Infer-the-Model)
9. [Show predictions](#Show-Predictions)

### 4.1. Import Required Modules <a name="Import-Required-Modules"></a>

Import the Python* modules that you will use in the sample code:
- [pathlib](https://docs.python.org/3/library/os.html#module-os) is a standard Python module used for filename parsing.
- [cv2](https://docs.opencv.org/trunk/) is an OpenCV module used to work with images.
- [NumPy](http://www.numpy.org/) is an array manipulation module used to process images as arrays.
- [OpenVINO Inference Engine](https://docs.openvinotoolkit.org/latest/openvino_docs_IE_DG_Deep_Learning_Inference_Engine_DevGuide.html) is an OpenVINO™ Python API module used for inference.
- [IPython](https://ipython.readthedocs.io/en/stable/index.html) is an IPython API uused for showing images and videos in the notebook

Run the cell below to import the modules. 

In [ ]:
import os
import cv2
import numpy as np
from openvino.inference_engine import IECore
from IPython.display import HTML, Image, display
from matplotlib import pyplot as plt

%matplotlib inline

### 4.2. Configure an Inference <a name="Configure-an-Inference"></a>

Once you have the OpenVINO™ IR of your model, you can start experimenting with it by inferring it and inspecting its output. 

> **NOTE**: Copy the paths to the `.xml` and `.bin` files from the DL Workbench UI and paste them below.

![](pictures/model-paths.png)

#### Required parameters

Parameter| Explanation
---|---
**model_xml**| Path to the `.xml` file of OpenVINO™ IR of your model
**model_bin**| Path to the `.bin` file of OpenVINO™ IR of your model

In [ ]:
# Model IR files
face_detection_model_xml = 'data/models/face-detection-adas-0001.xml'
face_detection_model_bin = 'data/models/face-detection-adas-0001.bin'

#### Optional Parameters

Experiment with optional parameters after you go the full workflow of the tutorial.

Parameter| Explanation
---|---
**input_image_path**| Path to an input image. Use the `car.bmp` image placed in the directory of the notebook or, if you have imported a dataset in the DL Workbench, copy the path to an image in the dataset.
**device**| Specify the [target device](https://docs.openvinotoolkit.org/latest/workbench_docs_Workbench_DG_Select_Environment.html) to infer on: CPU, GPU, or MYRIAD. Note that the device must be present. For this tutorial, use `CPU` which is known to be present.
**prob_threshold**| Probability threshold in percent to filter detection results 

In [ ]:
# Input image file. 
input_image_path = 'data/input_image.jpg'

# Device to use
device = 'CPU'

# Minimum percentage threshold to detect an object
prob_threshold = 50

print(
f'''Configuration parameters settings:
    model_xml={face_detection_model_xml},
    model_bin={face_detection_model_bin},
    input_image_path={input_image_path},
    device={device}, 
    prob_threshold={prob_threshold}''',
)

### 4.3 Initialize the OpenVINO™ Runtime <a name="#Initialize-the-OpenVINO™-Runtime"></a>

Once you define the parameters, let's initiate the `IECore` object that accesses OpenVINO™ runtime capabilities.

In [ ]:
# Create an Inference Engine instance
ie_core = IECore()

### 4.4 Read the Model <a name="#Read-the-Model"></a>

Put the IR of your model in the memory.

In [ ]:
# Read the model from IR files
face_detection_model = ie_core.read_network(model=face_detection_model_xml, weights=face_detection_model_bin)

### 4.5 Make the Model Executable <a name="#Make-the-Model-Executable"></a>

Reading a network is not enough to start a model inference. The model must be loaded to a particular abstraction representing a particular accelerator. In OpenVINO™, this abstraction is called *plugin*. A network loaded to a plugin becomes executable and will be inferred in one of the next steps. 

After loading, we keep necessary model information such as names of input and output blobs: `input_blob` and `output_blob`. Let's remember the input dimensions of your model:
- `n` - input batch size
- `c` - number of input channels. Often, it is `1` or `3`, which means that the model expects either a grayscale or a color image.
- `h` - input image height
- `w` - input image width

In [ ]:
# Store names of input and output of the model
face_detector_input_name = next(iter(face_detection_model.input_info))
face_detector_output_name = next(iter(face_detection_model.outputs))

# Read the input dimensions: n=batch size, c=number of channels, h=height, w=width
face_detector_input_shape = face_detection_model.input_info[face_detector_input_name].input_data.shape
n, c, face_detector_input_height, face_detector_input_width = face_detector_input_shape

print(f'Face Detection model input dimensions: n={n}, c={c}, h={face_detector_input_height}, w={face_detector_input_width}')

<img src="pictures/face_detection.png" width="1050" />

### 4.6 Load the Model to the Device <a name="#Load-to-Device"></a>

This step is necessary to infer the model. You can learn more in the [documentation](https://docs.openvinotoolkit.org/latest/ie_python_api/classie__api_1_1IECore.html#ac9a2e043d14ccfa9c6bbf626cfd69fcc)

In [ ]:
face_detector = ie_core.load_network(network=face_detection_model, device_name=device)
print(f'Loaded the model into the {device} device.')

### 4.7 Prepare an Image for Model Inference <a name="#Prepare-an-Image-for-Model-Inference"></a>

Now let's read and prepare the input image by resizing and re-arranging its dimensions according to the input dimensions of the model.

In [ ]:
# Define the function to load the input image
def load_input_image(input_path):   
    # Use OpenCV to load the input image
    return cv2.imread(input_path)

# Define the function to pre-process (resize, transpose) the input image
def pre_process_input_image(image: np.ndarray, target_height: int, target_width: int) -> np.ndarray:
    # Resize the image dimensions from image to model input w x h
    resized_image = cv2.resize(image, (target_width, target_height))
    
    # Change data layout from HWC to CHW
    transposed_image = resized_image.transpose((2, 0, 1))
    
    n = 1 # Batch is always 1 in our case
    c = 3 # Channels is always 3 in our case
    
    # Reshape to input dimensions
    reshaped_image = transposed_image.reshape((n, c, target_height, target_width))
    return reshaped_image

# Define the function to show an image
def show_image(image: np.ndarray):
    plt.axis('off')
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    # Encode ndarray
#     _, data = cv2.imencode('.jpg', image) 
    
    # Create IPython.display.Image instance to display the image in the notebook
#     image = Image(data=data)

#     display(image)

In [ ]:
original_image = load_input_image(input_image_path)
show_image(original_image)

In [ ]:
# Save the shape of the original image to scale inference results in the feature
original_image_height, original_image_width, *_ = original_image.shape

# Prepare the image
input_frame = pre_process_input_image(original_image, target_height=face_detector_input_height, target_width=face_detector_input_width)

# Display the input image
show_image(input_frame)

### 4.8 Infer the Model <a name="#Infer-the-Model"></a>

Now let's proceed to the inference by feeding the prepared image to the model.

In [ ]:
face_detection_inference_results = face_detector.infer(
    inputs={
        face_detector_input_name: input_frame
    }
)

Then we print the inference results which is a dictionary where the keys are the names of the outputs and the values are the inference results for each output layer.

In [ ]:
print(face_detection_inference_results)

Get the needed inference results for single output.

In [ ]:
face_detection_inference_results = face_detection_inference_results[face_detector_output_name]

### 4.9 Show Predictions <a name="#Show-Predictions"></a>

The next step is to parse the inference results and draw boxes over the objects detected in the image.

A result of model inference (`face_detection_inference_results`) is an array of predictions. Each prediction `object` has a following structure:

- `object[2]`: Confidence level that currently detected object is an instance of the predicted class
- `object[3]`: lower x coordinate of the detected object 
- `object[4]`: lower y coordinate of the detected object
- `object[5]`: upper x coordinate of the detected object
- `object[6]`: upper y coordinate of the detected object

![](pictures/models_diagram.png)

The function of converting the initial results of the face detector into a convenient structure.

In [ ]:
def parse_face_detection_results(inference_results: np.ndarray, original_image_width: int, original_image_height: int) -> list:
    # Prepare list to save detected faces 
    detected_faces = []
    
    # Iterate through all detected faces
    for inference_result in inference_results[0][0]:
        
        # Get the probability of the detected face and convert it to percent
        probability = inference_result[2]
        confidence = round(probability * 100, 1)

        # If confidence is more than the specified threshold, draw and label the box 
        if confidence < prob_threshold:
            continue

        # Get coordinates of the box containing the detected object
        xmin = int(inference_result[3] * original_image_width)
        ymin = int(inference_result[4] * original_image_height)
        xmax = int(inference_result[5] * original_image_width)
        ymax = int(inference_result[6] * original_image_height)

        detected_face = (xmin, ymin, xmax, ymax, confidence)
        detected_faces.append(detected_face)
            
    return detected_faces

Parse faces.

In [ ]:
# Parse detected faces from inference results 
detected_faces = parse_face_detection_results(face_detection_inference_results, original_image_width, original_image_height)
detected_faces

The function to process inference results of the face detector - draw boxes in the original image.

In [ ]:
# Function to process inference results of the face detector 
def draw_boxes_of_detected_faces(detected_faces: list, original_image: np.ndarray) -> np.ndarray:
    processed_image = original_image.copy()
    
    # Get output results
    color = (12.5, 255, 255)
    
    # Loop through all possible results
    for detected_face in detected_faces:
        xmin, ymin, xmax, ymax, confidence = detected_face

        # Draw the box and label for the detected object
        cv2.rectangle(processed_image, (xmin, ymin), (xmax, ymax), color, 4)
        cv2.putText(processed_image, 
                    f'{confidence} %', (xmin, ymin - 7), 
                    cv2.FONT_HERSHEY_COMPLEX, 1, color, 2)
            
    return processed_image

In [ ]:
# Draw boxes of the detected faces in the image
processed_image = draw_boxes_of_detected_faces(detected_faces, original_image)

show_image(processed_image)

# Practice <a name="tasks"></a>

1. [Task 1: Apply predefined blur method to the given image at inferred coordinates](#task1) 
2. [Task 2: Add blurring logic to pre-defined video processor](#task2)
3. [Task 3: Replace each face on the photo with a smile with corresponding emotion](#task3) 
4. [Task 4: Put Smiles on Top of the Face in the Video](#task4)

## Task 1: Apply predefined blur method to the given image at inferred coordinates <a name="#task1"></a>

The goal of this task is to blur all detected faces on the photo. We prepared the functions to process detected faces and blur the faces. You need to fill the part by cutting the face from the image, blur it, and past the blurry image to the image.

First of all, define the function to blur a part of the image.

In [ ]:
def blur(image: np.ndarray) -> np.ndarray:
    height, width = image.shape[:2]
    pixels_count = 16
    
    # Resize the image to pixels_count*pixels_count with interpolation to blur the image
    resized_image = cv2.resize(image, (pixels_count, pixels_count), interpolation=cv2.INTER_LINEAR)
    # Resize the image to original image size
    return cv2.resize(resized_image, (width, height), interpolation=cv2.INTER_NEAREST)

In [ ]:
# Load an example image 
example_image = load_input_image('pictures/Tarkan.jpg')
print('Original image:')
show_image(example_image)

# Blur the image
print('Blurry image:')
blurry_image = blur(example_image)
show_image(blurry_image)

In [ ]:
# Load an example image 
example_image = load_input_image('pictures/Tarkan.jpg')
print('Original image:')
show_image(example_image)

# blur the image
print('Blurry image:')
blurry_image = blur(example_image)
show_image(blurry_image)

Now define the function for the inference. This function must prepare image for inference (use `pre_process_input_image` for this) and run inference of the image using  `face_detectior`. Note that this function will also be used later. 

In [ ]:
def face_detector_inference(image: np.ndarray)-> np.ndarray:
    # 1. Prepare the image
    input_frame = pre_process_input_image(image, target_width=face_detector_input_width, target_height=face_detector_input_height)

    # 2. Infer the model
    face_detection_inference_results = face_detector.infer(inputs={face_detector_input_name: input_frame})  
    return face_detection_inference_results[face_detector_output_name]

Infer the model on the given image.

In [ ]:
inference_result = face_detector_inference(original_image)

Then define the function to process inference results of the face detection model. Use the `blur` function to blur the part of the image where the face is located.

In [ ]:
def blur_faces(face_detection_inference_result: np.ndarray, original_image: np.ndarray) -> np.ndarray:
    # Get original image size  
    original_image_height, original_image_width,  _ = original_image.shape
    
    # Prepare result image to not affect the original image
    processed_image = original_image.copy()
    
    
    detected_faces = parse_face_detection_results(face_detection_inference_result, original_image_width, original_image_height)

    for detected_face in detected_faces:
        xmin, ymin, xmax, ymax, _ = detected_face
        # Get the face from the original image 
        face = original_image[ymin:ymax, xmin:xmax]
        # Blur face using blur function 
        blurry_face = blur(face)
        # replace the face to the blurry face in the result image 
        processed_image[ymin:ymax, xmin:xmax] = blurry_face
            
    return processed_image

In [ ]:
def blur_faces(face_detection_inference_result: np.ndarray, original_image: np.ndarray) -> np.ndarray:
    # Get original image size  
    original_image_height, original_image_width,  _ = original_image.shape
    
    # Prepare result image to not affect the original image
    processed_image = original_image.copy()
    
    
    # 1. Parse model inference results
    detected_faces = # ...
    
    # 2. Iterate through faces and blur each face
    for detected_face in detected_faces:
        pass
            
    return processed_image

Run face detector inference and blur faces on the image.

In [ ]:
# 3. Blur faces on image
processed_image = blur_faces(inference_result, original_image)

show_image(processed_image)

## Task 2: Add blurring logic to pre-defined video processor <a name="#task2"></a>

Now that we have learned how to blur faces on a image, let's try to blur all faces in a video. We will use the previously defined functions to do this. 

Define paths to input and output videos.

In [ ]:
# Input video file
input_video_path = 'data/input.mp4'

# Output video file
output_video_path = 'data/output.mp4'

Open the input video.

In [ ]:
input_video_stream = cv2.VideoCapture(input_video_path)

Create output video stream.

In [ ]:
def prepare_output_video_stream(input_video_stream: cv2.VideoCapture, output_video_file_path: str) -> cv2.VideoWriter:
    width  = int(input_video_stream.get(3))
    height = int(input_video_stream.get(4))
    video_writer = cv2.VideoWriter(output_video_file_path, cv2.VideoWriter_fourcc(*'avc1'), 20, (width, height))
    return video_writer

output_video_stream = prepare_output_video_stream(input_video_stream, output_video_path)

In [ ]:
while input_video_stream.isOpened():
    # 1. Read the next frame from the input video 
    return_code, original_frame = input_video_stream.read()
    if not return_code:
        break
        
    inference_result = face_detector_inference(original_frame)

    # 3. Blur faces on image
    processed_image = blur_faces(inference_result, original_frame)
    
    # 3. Write the resulting frame to the output stream
    output_video_stream.write(processed_image)
    

input_video_stream.release()
# Save the resulting video
output_video_stream.release()

In [ ]:
while input_video_stream.isOpened():
    # 1. Read the next frame from the input video 
    return_code, original_frame = input_video_stream.read()
    if not return_code:
        break
        
    
    # 2. Infer face detector
    inference_result = # ...

    # 3. Blur faces on image
    processed_image = # ...
    
    # 3. Write the resulting frame to the output stream
    output_video_stream.write(processed_image)
    

input_video_stream.release()
# Save the resulting video
output_video_stream.release()

In [ ]:
# Show a source video
HTML(f"""<video width="600" height="400" controls><source src="{output_video_path}" type="video/mp4"></video>""")

## Task 3: Replace each face on the photo with a smile with corresponding emotion <a name="#task3"></a>

###  Download a Pretrained Model from the Open Model Zoo

OpenVINO™ toolkit includes the [Model Optimizer](http://docs.openvinotoolkit.org/latest/_docs_MO_DG_Deep_Learning_Model_Optimizer_DevGuide.html) used to convert and optimize trained models into Intermediate Representation (IR) model files, and the [Inference Engine](http://docs.openvinotoolkit.org/latest/_docs_IE_DG_Deep_Learning_Inference_Engine_DevGuide.html), which uses the IR model files to run an inference on hardware devices. The IR model files are created from models trained in popular frameworks, like Caffe\*, TensorFlow\*, and others. 

OpenVINO™ [Model Downloader](http://docs.openvinotoolkit.org/latest/_tools_downloader_README.html) downloads common inference models from the [Intel® Open Model Zoo](https://github.com/opencv/open_model_zoo). 

Let's download the `emotions-recognition-retail-0003` model first.

In [ ]:
!python3 /opt/intel/openvino_2021/deployment_tools/open_model_zoo/tools/downloader/downloader.py --name emotions-recognition-retail-0003 --precision FP16 --output_dir data/model

In [ ]:
# Model IR files
emotion_recognition_model_xml = 'data/model/intel/emotions-recognition-retail-0003/FP16/emotions-recognition-retail-0003.xml'
emotion_recognition_model_bin = 'data/model/intel/emotions-recognition-retail-0003/FP16/emotions-recognition-retail-0003.bin'

### Read the Model 

Call ie_core.read_network to read the OpenVINO IR model. The method has two arguments:
    
    1. model - path to the xml model file
    2. weights - path to the bin model file

In [ ]:
emotion_recognition_model = ie_core.read_network(model=emotion_recognition_model_xml, weights=emotion_recognition_model_bin)

### Get Input Name and Shape of the Model 

Get the input name and shape of the emotion recognizer. For more information about the model, refer the [documentation](https://docs.openvinotoolkit.org/latest/omz_models_model_emotions_recognition_retail_0003.html)

In [ ]:
emotion_recognizer_input_name = next(iter(emotion_recognition_model.input_info))

emotion_recognizer_input_shape = emotion_recognition_model.input_info[emotion_recognizer_input_name].input_data.shape
n, c, emotion_recognizer_input_height, emotion_recognizer_input_width = emotion_recognizer_input_shape

print(f'Input shape of the emotion recognition network is n={n}, c={c}, h={emotion_recognizer_input_height}, w={emotion_recognizer_input_width}')

In [ ]:
emotion_recognizer_output_name = next(iter(emotion_recognition_model.outputs))

<img src="pictures/emotion_recognition.png" width="1050" />

### Step 4: Load the Network to the Device

Use the instance of `IECore`.
The class `IECore` has a special function called `load_network`, which loads a network to a device.
This function prepares the network for the first inference on the device 
and returns an instance of the network prepared for an inference (execution). 
This function has many parameters, but in this case, you need to know only about two of them:
* `network` - instance of `IENetwork`
* `device_name` - string, contains a device name to infer a model on: CPU, GPU and so on.

In [ ]:
emotion_recognizer = ie_core.load_network(emotion_recognition_model, device)

### Step 5: Prepare a Function for Inference

In [ ]:
def emotion_recognizer_inference(face_frame: np.ndarray):
    prepared_frame = pre_process_input_image(face_frame, target_width=emotion_recognizer_input_width, target_height=emotion_recognizer_input_height)
    
    # Run the inference how you did it early
    inference_results = emotion_recognizer.infer({
        emotion_recognizer_input_name: prepared_frame
    })
    
    # For better understanding of model inference result, refer to the documentation 
    # 
    return inference_results[emotion_recognizer_output_name]

Let's take a closer look at the model. Run single inference of the image.

In [ ]:
original_frame = load_input_image('./data/Yarik.jpg')

# Infer the model
face_detection_inference_results = emotion_recognizer_inference(original_frame)
face_detection_inference_results

The inference results is a nested list with prediction for emotions:

    - neutral
    - happy
    - sad
    - surprised
    - angry
    
 Lets process this results for the image:

In [ ]:
# Flat the nesatead list
face_detection_inference_result = face_detection_inference_results.flatten()

# Define the emotions
emotions = ['neutral', 'happy', 'sad', 'surprised', 'angry']

show_image(original_frame)

print('Inference results:')
for index, prediction in enumerate(face_detection_inference_result):
    emotion = emotions[index]
    print(f'{emotion:10}:\t{prediction}')

### Step 6: Replace the Face with a Corresponding Smile

Now we are ready to replace the detected face with a smile that corresponds to the emotion:

In [ ]:
# Load the image
original_image = load_input_image(input_image_path)
original_image_height, original_image_width, *_ = original_image.shape

# Display the input image
print("Input image:")
show_image(original_image)

Prepare the function that returns a coresponding smile.

In [ ]:
def get_smile_by_index(emotion_inference_result: np.ndarray) -> np.ndarray:
    emotions = ['neutral', 'happy', 'sad', 'surprise', 'anger']
    # Get the index of the emotion with the highest probability 
    emotion_index = np.argmax(emotion_inference_result.flatten()) 
    smile_path = f'./data/{emotions[emotion_index]}.png'
    return cv2.imread(smile_path, -1)

For the given inference result, we will get the following smile:

In [ ]:
smile = get_smile_by_index(face_detection_inference_result)
show_image(smile)

Define the function that puts a smile on top of the face.

In [ ]:
def put_smile_on_top_of_face(image: np.ndarray, inference_results: np.ndarray, face_xmin:int, face_ymin:int, face_xmax:int, face_ymax:int):

    # Get width and height of the face 
    width = xmax - xmin
    height = ymax - ymin
    
    # Get smile by inference results
    smile = get_smile_by_index(inference_results)
    
    # Resize the smile to face shape
    resized_smile = cv2.resize(smile, (width, height))

    # Put the smile on top of the face
    alpha_s = resized_smile[:, :, 3] / 255.0
    alpha_l = 1.0 - alpha_s
    for c in range(0, 3):
        image[ymin:ymax, xmin:xmax, c] = (alpha_s * resized_smile[:, :, c] + alpha_l * image[ymin:ymax, xmin:xmax, c])


Run face detector inference to detect faces on the image.

In [ ]:
face_detection_inference_results = face_detector_inference(original_image)

# Parse detected faces
faces_coordinates = parse_face_detection_results(face_detection_inference_results, original_image_width, original_image_height)

Put smiles on top of the detected faces.

In [ ]:
# Prepare the result image
processed_image = original_image.copy()

for face_coordinates in faces_coordinates:
    xmin, ymin, xmax, ymax, confidence = face_coordinates
    
    # Get the face from the image
    face = original_image[ymin:ymax, xmin:xmax]

    # Run emotion recognizer 
    emotion_predictions = emotion_recognizer_inference(face)
    
    # Replace the face with the smile
    put_smile_on_top_of_face(processed_image, emotion_predictions, xmin, ymin, xmax, ymax)

In [ ]:
# Prepare the result image
processed_image = original_image.copy()

for face_coordinates in faces_coordinates:
    xmin, ymin, xmax, ymax, confidence = face_coordinates
    
    # Get the face from the image
    face = # ... 

    # Run emotion recognizer 
    emotion_predictions = # ...
    
    # Replace the face with the smile
    put_smile_on_top_of_face(...)

In [ ]:
# Prepare the result image
processed_image = original_image.copy()

for face_coordinates in faces_coordinates:
    xmin, ymin, xmax, ymax, confidence = face_coordinates
    
    # Get the face from the image
    face = # ... 

    # Run emotion recognizer 
    emotion_predictions = # ...
    
    # Replace the face with the smile
    put_smile_on_top_of_face(...)

Show the result image.

In [ ]:
show_image(processed_image)

## Task 4: Put Smiles on Top of the Face in the Video <a name="#task4"></a>

In [ ]:
input_video_stream = cv2.VideoCapture(input_video_path)

original_video_width = int(input_video_stream.get(3))
original_video_height = int(input_video_stream.get(4))

In [ ]:
output_video_stream = prepare_output_video_stream(input_video_stream, output_video_path)

In [ ]:
while input_video_stream.isOpened():
    # 1. Read the next frame from the input video 
    finish, original_frame = input_video_stream.read()
    if not finish:
        break
        
    # 2. apply face replacement from previous step
    face_detection_inference_results = face_detector_inference(original_frame)
    
    faces_coordinates = parse_face_detection_results(face_detection_inference_results, original_video_width, original_video_height)

    for face_coordinates in faces_coordinates:

        xmin, ymin, xmax, ymax, _ = face_coordinates
        face = original_frame[ymin:ymax, xmin:xmax]

        emotion_predictions = emotion_recognizer_inference(face)

        put_smile_on_top_of_face(original_frame, emotion_predictions, xmin, ymin, xmax, ymax)
    # 3. Write the resulting frame to the output stream
    output_video_stream.write(original_frame)
    
input_video_stream.release()
# Save the resulting video
output_video_stream.release()

In [ ]:
while input_video_stream.isOpened():
    # 1. Read the next frame from the input video 
    finish, original_frame = input_video_stream.read()
    if not finish:
        break
    
    # 2. Your turn - implement pipeline
    
    # 3. Write the resulting frame to the output stream
    output_video_stream.write(original_frame)
    
input_video_stream.release()
# Save the resulting video
output_video_stream.release()

In [ ]:
# Show a source video
HTML(f"""<video width="600" height="400" controls><source src="{output_video_path}" type="video/mp4"></video>""")

# Call to action #1. Try Workbench right now: locally or in Dev Cloud

# Call to action #2. Try SberCloud with oneAPI and OpenVINO

![](pictures/one_API.jpeg)

# Call to action #1. Try Workbench right now: locally or in Dev Cloud

# Call to action #2. Try SberCloud with oneAPI and OpenVINO

![](pictures/one_API.jpeg)

# Call to action #3. [Learn OpenVINO Toolkit on the Coursera](https://www.coursera.org/learn/intel-openvino)

![](pictures/ov-coursera.png)

# Call to action #4. [Explore OpenVINO Github Repository](https://github.com/openvinotoolkit/openvino)

![](pictures/ov-repository.png)